<a href="https://colab.research.google.com/github/tatendatobaiwa/cnn/blob/main/CNN_Medical_Imaging_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***SIMPLE CNN MODEL BUILD FOR LUNG CANCER IMAGING***


**Step 1: Mount Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Step 2: Create Directories in Drive**

**Step 3: Install Dependenices**

In [2]:
!pip install -q tensorflow==2.13.0
!pip install -q tensorflow-addons==0.23.0
!pip install albumentations==1.3.0
!pip install numpy==1.24.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.2/524.2 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 27.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic-core 2.27.2 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
openai 1.61.1 requires typing-extensions<5,>=4.11, but you have typing-extensions 4.5.0 which is incompatible.
nibabel 5.3.2 requires typing-extensions>=4.6; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
sqlalchemy 2.0.38 requires typing-extensions>=4.6.0, but you hav

In [3]:
import tensorflow as tf
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import os
import sys
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

project_path = '/content/drive/MyDrive/CNN_Medical_Imaging_Project'
sys.path.append(project_path)

**Mini Step 3a: Verify Fixes**

In [4]:
import tensorflow as tf
import tensorflow_addons as tfa

print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow Addons version: {tfa.__version__}")

TensorFlow version: 2.13.0
TensorFlow Addons version: 0.23.0


/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


**Step 4: data_processing.py: Load and Preprocess Data**

In [5]:
import cv2
import numpy as np
from pathlib import Path
import albumentations as A
from tensorflow.keras.utils import Sequence

class AlbumentationsSequence(Sequence):
    """Custom Sequence class for data augmentation with Albumentations."""
    def __init__(self, X, y, batch_size, transform):
        self.X = X  # List or array of uint8 images
        self.y = y  # Array of integer labels (or None for test data)
        self.batch_size = batch_size
        self.transform = transform

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, idx):
        batch_X = self.X[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_X_aug = [self.transform(image=x)['image'] for x in batch_X]
        batch_X_aug = np.array(batch_X_aug)
        batch_X_aug = np.stack((batch_X_aug,) * 3, axis=-1)  # Convert grayscale to 3-channel RGB
        if self.y is not None:
            batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
            return batch_X_aug, batch_y
        return batch_X_aug

class DataProcessor:
    """Class to handle loading and preprocessing of image data."""
    def __init__(self, target_size=(224, 224)):
        self.target_size = target_size
        self.class_names = ['benign', 'malignant', 'normal']

    def load_images(self, data_dir):
        """Load images and labels from subdirectories (for training and validation)."""
        data_dir = Path(data_dir)
        images = []
        labels = []
        for label, class_name in enumerate(self.class_names):
            class_dir = data_dir / class_name
            if not class_dir.exists():
                print(f"Warning: Directory {class_dir} does not exist.")
                continue
            img_paths = list(class_dir.glob('*'))
            for img_path in img_paths:
                if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        img = cv2.resize(img, self.target_size)
                        images.append(img)
                        labels.append(label)
        if not images:
            raise ValueError(f"No valid images found in {data_dir}")
        images = np.array(images)
        labels = np.array(labels, dtype=np.int32)
        print(f"Loaded {len(images)} images from {data_dir}")
        return images, labels, self.class_names

    def load_test_images(self, test_dir):
        """Load test images from a flat directory (no labels)."""
        test_dir = Path(test_dir)
        images = []
        img_paths = list(test_dir.glob('*'))
        for img_path in img_paths:
            if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, self.target_size)
                    images.append(img)
        if not images:
            raise ValueError(f"No valid images found in {test_dir}")
        images = np.array(images)
        print(f"Loaded {len(images)} test images from {test_dir}")
        return images

    def create_generators(self, X_train, y_train, X_val, y_val, batch_size=32):
        """Create training and validation data generators with augmentations."""
        train_transform = A.Compose([
            A.Rotate(limit=20, p=0.5),
            A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=0, p=0.5),
            A.HorizontalFlip(p=0.5),
            A.ElasticTransform(alpha=34, sigma=4, p=0.3),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.Normalize(mean=0, std=1),
        ])
        val_transform = A.Compose([
            A.Normalize(mean=0, std=1),
        ])
        train_gen = AlbumentationsSequence(X_train, y_train, batch_size, train_transform)
        val_gen = AlbumentationsSequence(X_val, y_val, batch_size, val_transform)
        return train_gen, val_gen

    def create_test_generator(self, X_test, batch_size=32):
        """Create a test data generator (no labels)."""
        test_transform = A.Compose([
            A.Normalize(mean=0, std=1),
        ])
        test_gen = AlbumentationsSequence(X_test, None, batch_size, test_transform)
        return test_gen

**Step 5: train.py: Model Training**

In [6]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import numpy as np
from data_processing import DataProcessor
from model_mobilenet import create_mobilenet_model

def train_model(train_gen, val_gen, class_names, epochs=50):
    """Train and fine-tune the model with class weights and callbacks."""
    # Ensure train_gen.y is a NumPy array of integers
    train_labels = np.array(train_gen.y, dtype=np.int32)
    if len(train_labels) == 0:
        raise ValueError("No training labels available in train_gen.")

    # Compute class weights to handle imbalance
    classes = np.unique(train_labels)
    if len(classes) == 0:
        raise ValueError("No unique classes found in training labels.")
    class_weights = compute_class_weight('balanced', classes=classes, y=train_labels)
    class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
    print(f"Class weights: {class_weights_dict}")

    # Create and compile the model
    model = create_mobilenet_model(num_classes=len(class_names))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Define callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Initial training
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[early_stop, checkpoint, reduce_lr],
        class_weight=class_weights_dict
    )

    # Fine-tuning: Unfreeze layers from block_13_expand
    base_model = model.layers[1]  # MobileNetV2 base model layer
    set_trainable = False
    for layer in base_model.layers:
        if layer.name == 'block_13_expand':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

    # Recompile with lower learning rate for fine-tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Fine-tuning training
    history_fine = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[early_stop, checkpoint, reduce_lr],
        class_weight=class_weights_dict
    )

    return model, history, history_fine

if __name__ == "__main__":
    processor = DataProcessor()

    # Load and split training data
    X, y, class_names = processor.load_images('/content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/train')
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    print(f"Training data: {X_train.shape}, {y_train.shape}")
    print(f"Validation data: {X_val.shape}, {y_val.shape}")

    # Create generators
    train_gen, val_gen = processor.create_generators(X_train, y_train, X_val, y_val)

    # Train the model
    model, history, history_fine = train_model(train_gen, val_gen, class_names)

    # Load and predict on test data
    X_test = processor.load_test_images('/content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/test')
    test_gen = processor.create_test_generator(X_test)
    if len(X_test) > 0:
        y_pred = model.predict(test_gen)
        y_pred_classes = np.argmax(y_pred, axis=1)
        print("Predicted classes for test images:", y_pred_classes)
    else:
        print("No test data loaded. Please check the test directory.")

Loaded 871 images from /content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/train
Training data: (696, 224, 224), (696,)
Validation data: (175, 224, 224), (175,)
Class weights: {0: 2.4166666666666665, 1: 0.8656716417910447, 2: 0.6987951807228916}
9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/50
22/22 [==============================] - ETA: 0s - loss: 1.0583 - accuracy: 0.5273

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


22/22 [==============================] - 36s 1s/step - loss: 1.0583 - accuracy: 0.5273 - val_loss: 0.5914 - val_accuracy: 0.7257 - lr: 0.0010
Epoch 2/50
22/22 [==============================] - 39s 2s/step - loss: 0.6859 - accuracy: 0.6968 - val_loss: 0.4801 - val_accuracy: 0.7886 - lr: 0.0010
Epoch 3/50
22/22 [==============================] - 28s 1s/step - loss: 0.6051 - accuracy: 0.7443 - val_loss: 0.6399 - val_accuracy: 0.6171 - lr: 0.0010
Epoch 4/50
22/22 [==============================] - 29s 1s/step - loss: 0.6088 - accuracy: 0.7198 - val_loss: 0.3174 - val_accuracy: 0.8686 - lr: 0.0010
Epoch 5/50
22/22 [==============================] - 33s 2s/step - loss: 0.5344 - accuracy: 0.7328 - val_loss: 0.3520 - val_accuracy: 0.8286 - lr: 0.0010
Epoch 6/50
22/22 [==============================] - 29s 1s/step - loss: 0.5510 - accuracy: 0.7744 - val_loss: 0.2832 - val_accuracy: 0.9086 - lr: 0.0010
Epoch 7/50
22/22 [==============================] - 29s 1s/step - loss: 0.5283 - accuracy: 0.

AttributeError: 'Conv2D' object has no attribute 'layers'

**Step 6: evaluate.py: Model Evaluation**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import label_binarize

def evaluate_model(model, X_test, y_test, class_names, save_path=None):
    """
    Evaluate a trained model on test data with classification metrics and visualizations.

    Parameters:
    - model: Trained Keras model.
    - X_test: Test images (numpy array).
    - y_test: True labels (numpy array).
    - class_names: List of class names (e.g., ['benign', 'malignant', 'normal']).
    - save_path: Optional path to save confusion matrix plot (e.g., 'confusion_matrix.png').
    """
    # Predict probabilities and classes
    y_pred_probs = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)

    # Classification Report
    print("Classification Report:")
    report = classification_report(y_test, y_pred_classes, target_names=class_names)
    print(report)

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    if save_path:
        plt.savefig(save_path)
        print(f"Confusion matrix saved to {save_path}")
    plt.show()

    # Per-Class Sensitivity and Specificity
    print("\nPer-Class Metrics:")
    for i, class_name in enumerate(class_names):
        tp = cm[i, i]
        fn = np.sum(cm[i, :]) - tp
        fp = np.sum(cm[:, i]) - tp
        tn = np.sum(cm) - tp - fn - fp
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        print(f"{class_name}: Sensitivity = {sensitivity:.2f}, Specificity = {specificity:.2f}")

    # ROC-AUC Score
    y_test_bin = label_binarize(y_test, classes=range(len(class_names)))
    auc = roc_auc_score(y_test_bin, y_pred_probs, multi_class='ovr')
    print(f"\nROC-AUC Score (One-vs-Rest): {auc:.4f}")

if __name__ == "__main__":
    from tensorflow.keras.models import load_model
    from data_processing import DataProcessor

    # Load test data instead of validation data
    processor = DataProcessor()
    X_test, y_test, class_names = processor.load_test_images('/content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/test')

    # Load the trained model
    model = load_model('best_model.h5')

    # Evaluate the model
    evaluate_model(model, X_test, y_test, class_names, save_path='confusion_matrix.png')

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

**Step 7: xai.py: Implementation of Explainable AI (Grad-CAM)**

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from data_processing import DataProcessor

def get_gradcam_heatmap(model, img_array, layer_name, class_index):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_index]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def superimpose_heatmap(img, heatmap, alpha=0.4):
    """Superimpose the heatmap on the original image."""
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    img_rgb = np.stack((img,) * 3, axis=-1) * 255  # Convert grayscale to RGB
    superimposed_img = heatmap * alpha + img_rgb
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)
    return superimposed_img

def visualize_explanations(model, X, layer_name, num_images=5):
    test_gen = processor.create_test_generator(X)
    for i in range(min(num_images, len(X))):
        img = test_gen[i // test_gen.batch_size][i % test_gen.batch_size:i % test_gen.batch_size + 1]
        pred_probs = model.predict(img)
        pred_class = np.argmax(pred_probs)
        heatmap = get_gradcam_heatmap(model, img, layer_name, pred_class)
        superimposed_img = superimpose_heatmap(X[i], heatmap)
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(X[i], cmap='gray')
        plt.title(f'Predicted: {class_names[pred_class]}')
        plt.subplot(1, 2, 2)
        plt.imshow(superimposed_img)
        plt.title('Grad-CAM')
        plt.show()

if __name__ == "__main__":
    processor = DataProcessor()
    X_test = processor.load_test_images('/content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/test')
    model = load_model('best_model.h5')
    visualize_explanations(model, X_test, 'block_13_expand')